In [ ]:
# Train model
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
# from models import pretrained_model

In [ ]:
# Import saved model
from keras.models import load_model
model = load_model(r'D:\skin_disease_detection\backend\app\static\trained_models\skin_disease_model.h5')
print(model)

In [ ]:
import sys
sys.path.append(r'D:\skin_disease_detection\backend\app')
from services.training_ensemble_model_service import train_and_evaluate_ensemble
from utils.preprocess import get_data_generators
from tensorflow.keras.utils import to_categorical

# Call get_data_generators to load the data
train_generator, validation_generator, label_encoder = get_data_generators(
    metadata_path=r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_metadata.csv',
    target_size=(224, 224),
    batch_size=32
)

# Get one batch of data for training and validation
train_data, train_labels = next(train_generator)  # Retrieve one batch
val_data, val_labels = next(validation_generator)  # Retrieve one batch

# Check if the generator batch sizes match the expected number of classes
print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)

# One-hot encode the labels using the number of classes in label_encoder
num_classes = len(label_encoder.classes_)
train_labels = to_categorical(train_labels, num_classes=num_classes)
val_labels = to_categorical(val_labels, num_classes=num_classes)

# Assign validation data to test set (as per your setup)
test_data = val_data
test_labels = val_labels

# Confirm shapes before training
print("Train labels shape (one-hot):", train_labels.shape)
print("Validation labels shape (one-hot):", val_labels.shape)
print("Test data shape:", test_data.shape)
print("Test labels shape (one-hot):", test_labels.shape)

# Train and evaluate the ensemble model
# Ensure train_and_evaluate_ensemble is compatible with batch sizes and input structure
train_and_evaluate_ensemble(train_data, train_labels, val_data, val_labels, test_data, test_labels)


In [1]:
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Load the metadata
metadata = pd.read_csv('D:\\skin_disease_detection\\backend\\data\\Ham10000\\HAM10000_metadata.csv')
print(metadata['dx'].value_counts())  # This will show how many instances there are of each label

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.unique(metadata['dx']), 
    y=metadata['dx']
)
# Create a dictionary of class weights, mapping class labels to their corresponding weight
class_weights_dict = dict(zip(np.unique(metadata['dx']), class_weights))
print("Class weights:", class_weights_dict)

dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64
Class weights: {'akiec': np.float64(4.375273044997815), 'bcc': np.float64(2.78349082823791), 'bkl': np.float64(1.301832835044846), 'df': np.float64(12.440993788819876), 'mel': np.float64(1.2854575792581184), 'nv': np.float64(0.21338020666879728), 'vasc': np.float64(10.075452716297788)}
